In [ ]:
import tensorflow as tf

In [ ]:
!pip install -q transformers
!pip install -q sentencepiece
import torch
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score
import numpy as np
import pandas as pd


     |████████████████████████████████| 2.1MB 19.1MB/s 
     |████████████████████████████████| 3.3MB 49.4MB/s 
     |████████████████████████████████| 901kB 43.0MB/s 
     |████████████████████████████████| 1.2MB 17.5MB/s 


In [ ]:
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_test = pd.read_csv('drive/My Drive/Arabic_FakeNews/Model/Balance/DataText/PPV2test_news.csv')
df_test.text=df_test.text.astype(str)
df_test.head()

,text,label
0,فلكي يتوقع رمضان 30 يوما,1
1,روني واثق من الفوز على الجزائر,1
2,رئيس الوزراء الإيطالي بعد بيان الداخلية المصري...,1
3,معركة الديون . . من ربح ومن خسر ؟,1
4,وثائق بنما . . القضاء الفرنسي يبدأ جلسات محاكم...,1


In [ ]:
x_test = df_test.text
y_test = df_test.label

In [ ]:
from transformers import pipeline
Model='joeddav/xlm-roberta-large-xnli'
classifier = pipeline("zero-shot-classification", model=Model,tokenizer=Model)

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# First

## Sample Of Data

In [ ]:
subset_news=[]
for news in df_test.text[:5]:
  pred_zero=classifier(news, ["خبر حقيقي", "خبر كاذب"])
  print(pred_zero)
  if(pred_zero["scores"][0]> pred_zero["scores"][1]):
    subset_news.append(pred_zero["labels"][0])
    print(pred_zero["labels"][0])
  else:
    subset_news.append(pred_zero["labels"][1])
    print(pred_zero["labels"][1])

{'sequence': 'فلكي يتوقع رمضان 30 يوما', 'labels': ['خبر كاذب', 'خبر حقيقي'], 'scores': [0.6369655728340149, 0.3630343973636627]}
خبر كاذب
{'sequence': 'روني واثق من الفوز على الجزائر', 'labels': ['خبر كاذب', 'خبر حقيقي'], 'scores': [0.6110415458679199, 0.3889584243297577]}
خبر كاذب
{'sequence': 'رئيس الوزراء الإيطالي بعد بيان الداخلية المصرية : لن نقبل إلا بالحقيقة في مقتل جوليو ريجيني', 'labels': ['خبر حقيقي', 'خبر كاذب'], 'scores': [0.9926685094833374, 0.007331517059355974]}
خبر حقيقي
{'sequence': 'معركة الديون . . من ربح ومن خسر ؟', 'labels': ['خبر كاذب', 'خبر حقيقي'], 'scores': [0.7036762833595276, 0.2963237464427948]}
خبر كاذب
{'sequence': 'وثائق بنما . . القضاء الفرنسي يبدأ جلسات محاكمة " لوموند " في قضية بوتفليقة', 'labels': ['خبر كاذب', 'خبر حقيقي'], 'scores': [0.8064826130867004, 0.19351734220981598]}
خبر كاذب


In [ ]:
subset_news[:5]

['خبر كاذب', 'خبر كاذب', 'خبر حقيقي', 'خبر كاذب', 'خبر كاذب']

## All Data

In [ ]:
i=0
y_pred_new=[]
for news in df_test.text:
  i=i+1
  pred_zero=classifier(news, ["خبر حقيقي", "خبر كاذب"])
  if(pred_zero["scores"][0]> pred_zero["scores"][1]):
    y_pred_new.append(pred_zero["labels"][0])
  else:
    y_pred_new.append(pred_zero["labels"][1])
  
  print(str(i) +"-"+news)

y_pred_new[:5]

Streaming output truncated to the last 5000 lines.
8573-أوباما معلنا تأجيل سحب قواته : لن أسمح بتحويل أفغانستان إلى ملاذ آمن للجماعات الإرهابية
8574-جميل مطر يكتب : آن للحكام العرب أن يدركوا أن مصائر دولهم في مهب الريح
8575-وزارة العدل الفرنسية ترغب في تسهيل الطلاق وتسريعه وتقليل مصاريفه
8576-قانون " مقيد " للاتصالات يضع حكومة الكويت بمرمى انتقادات منظمات حقوقية
8577-التعذيب يعود في تونس الثورة
8578-قرار موريتانيا بحظر ارتداء " الدراعة " أثناء التدريس يثير غضب الأساتذة
8579-نزوح جماعي في شبيلي بالصومال
8580-استمرار فرز الأصوات في العراق
8581-سلطنة عمان تحتفل بعيدها ال 47
8582-مسؤول أمريكي سابق يبين لCNN مدى واقعية فكرة تحول داعش إلى دولة حقيقية . . ويؤكد : أهل الأنبار لا يريدون الدولة العراقية أو التنظيم
8583-منظمو مسابقة " ملكة جمال مصر " : تهدف للترويج السياحي ولم يعترض عليها رجال الدين
8584-قتيل و 4 جرحى بانفجار وسط القاهرة
8585-تعرفوا على 12 من أجمل لاعبي كأس العالم
8586-الحياة بشوارع دبي لحظة بلحظة مباشرة على خدمة غوغل للتجول الافتراضي
8587-قتيلان بتحطم طائرة شحن بدبي
8588-بان يدع

['خبر كاذب', 'خبر كاذب', 'خبر حقيقي', 'خبر كاذب', 'خبر كاذب']

## Accuercy

In [ ]:
convert = lambda x : 0 if x=='خبر كاذب' else 1

In [ ]:
y_pred=[]
for news in y_pred_new:
  y_pred.append(convert(news))

In [ ]:
y_pred[:5]

[0, 0, 1, 0, 0]

In [ ]:
print("Accuracy", accuracy_score(y_test, y_pred))
print("\nConfusion matrix\n",confusion_matrix(y_test,y_pred))
print("\nClassification report\n",classification_report(y_test,y_pred))

Accuracy 0.24410551134689065

Confusion matrix
 [[  306   109]
 [10150  3007]]

Classification report
               precision    recall  f1-score   support

           0       0.03      0.74      0.06       415
           1       0.97      0.23      0.37     13157

    accuracy                           0.24     13572
   macro avg       0.50      0.48      0.21     13572
weighted avg       0.94      0.24      0.36     13572



## Example

In [ ]:
#real
pred_zero=classifier('«خارج عن السيطرة».. صاروخ أطلقته الصين يوم الخميس قد يسقط على الأرض خلال أيام',["خبر حقيقي", "خبر كاذب"])

SCORES = pred_zero["scores"]
labels=pred_zero["labels"]
print(SCORES)
print(labels)

[0.7627281546592712, 0.23727187514305115]
['خبر كاذب', 'خبر حقيقي']


In [ ]:
#fake
pred_zero=classifier('✋✋ مين ضايل لم يزاود على السلطة و الاجهزة الامنية 👇👇👇 قناة 13 العبرية: زوجة منفذ عمـ ـلية زعتره منتصر شلبي أبرمت صفقة مع السفارة الأمريكية في القدس تضمنت تسليم زوجها مقابل عدم قتله، وهي من أعطت المعلومة عن مكان وجوده.وتسلم الايادي لوحوش البحرية وتحيا مصر',["خبر حقيقي", "خبر كاذب"])

SCORES = pred_zero["scores"]
labels=pred_zero["labels"]
print(SCORES)
print(labels)

[0.5945827960968018, 0.40541723370552063]
['خبر كاذب', 'خبر حقيقي']


In [ ]:
#fake
pred_zero=classifier('عااااااجل القوات البحرية وخفر السواحل تشتبه بإحدى مراكب الصيد بالمياه الإقليمية بالبحر الابيض وبحق التفتي والزيارة وجدت وسط العمال ضابط مخابرات تركي وسبع جنود وبتقنين الاجرائات تم اصطحابهم الي النيابة العسكرية مين هيقول لقردو مبروك وتسلم الايادي لوحوش البحرية وتحيا مصر',["خبر حقيقي", "خبر كاذب"])

SCORES = pred_zero["scores"]
labels=pred_zero["labels"]
print(SCORES)
print(labels)

[0.6703438758850098, 0.32965612411499023]
['خبر كاذب', 'خبر حقيقي']


# Second

## sample of Data

In [ ]:
subset_news=[]
for news in df_test.text[:5]:
  pred_zero=classifier(news, ["real news ", "fake news"])
  print(pred_zero)
  if(pred_zero["scores"][0]> pred_zero["scores"][1]):
    subset_news.append(pred_zero["labels"][0])
  else:
    subset_news.append(pred_zero["labels"][1])

subset_news[:5]

{'sequence': 'فلكي يتوقع رمضان 30 يوما', 'labels': ['fake news', 'real news '], 'scores': [0.5326231122016907, 0.4673768877983093]}
{'sequence': 'روني واثق من الفوز على الجزائر', 'labels': ['real news ', 'fake news'], 'scores': [0.6662662029266357, 0.3337337374687195]}
{'sequence': 'رئيس الوزراء الإيطالي بعد بيان الداخلية المصرية : لن نقبل إلا بالحقيقة في مقتل جوليو ريجيني', 'labels': ['real news ', 'fake news'], 'scores': [0.9963815212249756, 0.0036184450145810843]}
{'sequence': 'معركة الديون . . من ربح ومن خسر ؟', 'labels': ['real news ', 'fake news'], 'scores': [0.5518354177474976, 0.44816455245018005]}
{'sequence': 'وثائق بنما . . القضاء الفرنسي يبدأ جلسات محاكمة " لوموند " في قضية بوتفليقة', 'labels': ['fake news', 'real news '], 'scores': [0.7440835237503052, 0.25591644644737244]}


['fake news', 'real news ', 'real news ', 'real news ', 'fake news']

## All Data

In [ ]:
y_pred_new=[]
for news in df_test.text:
  pred_zero=classifier(news, ["real news ", "fake news"])
  if(pred_zero["scores"][0]> pred_zero["scores"][1]):
    y_pred_new.append(pred_zero["labels"][0])
  else:
    y_pred_new.append(pred_zero["labels"][1])

y_pred_new[:5]

['fake news', 'real news ', 'real news ', 'real news ', 'fake news']

## Accuercy

In [ ]:
convert = lambda x : 0 if x=='fake news' else 1

In [ ]:
y_pred=[]
for news in y_pred_new:
  y_pred.append(convert(news))
   

In [ ]:
y_pred[:5]

[0, 1, 1, 1, 0]

In [ ]:
print("Accuracy", accuracy_score(y_test, y_pred))
print("\nConfusion matrix\n",confusion_matrix(y_test,y_pred))
print("\nClassification report\n",classification_report(y_test,y_pred))

Accuracy 0.7186855290303567

Confusion matrix
 [[ 153  262]
 [3556 9601]]

Classification report
               precision    recall  f1-score   support

           0       0.04      0.37      0.07       415
           1       0.97      0.73      0.83     13157

    accuracy                           0.72     13572
   macro avg       0.51      0.55      0.45     13572
weighted avg       0.94      0.72      0.81     13572



## Sample

In [ ]:
pred_zero=classifier('«خارج عن السيطرة».. صاروخ أطلقته الصين يوم الخميس قد يسقط على الأرض خلال أيام', ["real news ", "fake news"])

SCORES = pred_zero["scores"]
labels=pred_zero["labels"]
print(SCORES)
print(labels)

[0.7357539534568787, 0.2642460763454437]
['fake news', 'real news ']


In [ ]:
pred_zero=classifier('✋✋ مين ضايل لم يزاود على السلطة و الاجهزة الامنية 👇👇👇 قناة 13 العبرية: زوجة منفذ عمـ ـلية زعتره منتصر شلبي أبرمت صفقة مع السفارة الأمريكية في القدس تضمنت تسليم زوجها مقابل عدم قتله، وهي من أعطت المعلومة عن مكان وجوده.وتسلم الايادي لوحوش البحرية وتحيا مصر', ["real news ", "fake news"])

SCORES = pred_zero["scores"]
labels=pred_zero["labels"]
print(SCORES)
print(labels)

[0.5521567463874817, 0.4478432238101959]
['real news ', 'fake news']


In [ ]:
pred_zero=classifier('عااااااجل القوات البحرية وخفر السواحل تشتبه بإحدى مراكب الصيد بالمياه الإقليمية بالبحر الابيض وبحق التفتي والزيارة وجدت وسط العمال ضابط مخابرات تركي وسبع جنود وبتقنين الاجرائات تم اصطحابهم الي النيابة العسكرية مين هيقول لقردو مبروك وتسلم الايادي لوحوش البحرية وتحيا مصر', ["real news ", "fake news"])

SCORES = pred_zero["scores"]
labels=pred_zero["labels"]
print(SCORES)
print(labels)

[0.6138929724693298, 0.3861069977283478]
['real news ', 'fake news']
